# Day 6: Function Calling - Teaching LLMs to Use Tools

## What You'll Learn Today

Today we dive deep into **function calling** - the mechanism that allows LLMs to use tools!

**Why Function Calling Matters:**
- Without it: LLM can only talk about using a weather API
- With it: LLM can actually CALL the weather API and get real data

### Today's Learning Path:
1. **Understand function calling** - How LLMs request tool use
2. **Function schemas** - Defining functions for the LLM
3. **Direct LLM function calling** - Without agents
4. **fncall_prompt_type** - 'qwen' vs 'nous' formats
5. **function_choice parameter** - Control when functions are called
6. **Parallel function calls** - Multiple tools at once
7. **Error handling** - Dealing with malformed calls

Let's unlock the full power of LLMs! 🔧

---
## Part 1: Configure Our Environment

Same Fireworks API setup.

In [ ]:
# ================================================
# FIREWORKS API CONFIGURATION  
# ================================================
import os
import json

os.environ['FIREWORKS_API_KEY'] = 'fw_3ZSpUnVR78vs38jJtyewjcWk'

llm_cfg_fireworks = {
    'model': 'accounts/fireworks/models/qwen3-235b-a22b-thinking-2507',
    'model_server': 'https://api.fireworks.ai/inference/v1',
    'api_key': os.environ['FIREWORKS_API_KEY'],
    'generate_cfg': {
        'max_tokens': 32768,
        'temperature': 0.6,
    }
}

llm_cfg = llm_cfg_fireworks

print('✅ Configured for Fireworks API')
print(f'   Model: Qwen3-235B-A22B-Thinking-2507')

---
## Part 2: What is Function Calling?

### The Function Calling Flow

Function calling is a **structured way for LLMs to request tool execution**:

```
1. User: "What's the weather in San Francisco?"
   ↓
2. You provide:
   - Message history
   - Available functions (with schemas)
   ↓
3. LLM responds with:
   {
     "function_call": {
       "name": "get_weather",
       "arguments": "{\"location\": \"San Francisco\"}"
     }
   }
   ↓
4. You execute the function
   ↓
5. You add the result to messages
   ↓
6. Call LLM again to get final answer
```

**Key insight:** The LLM doesn't execute functions - it just generates **structured requests** for YOU to execute!

### Function Schema Format

Functions are defined using **JSON Schema**:

```python
{
    'name': 'function_name',           # Unique identifier
    'description': 'What it does',     # How LLM knows when to use it
    'parameters': {                    # JSON Schema for parameters
        'type': 'object',
        'properties': {
            'param1': {
                'type': 'string',
                'description': 'What this parameter is for'
            }
        },
        'required': ['param1']         # Which params are mandatory
    }
}
```

---
## Part 3: Direct LLM Function Calling

### Example: Weather Function

Let's implement the classic weather example from the official Qwen-Agent examples!

In [ ]:
from qwen_agent.llm import get_chat_model
import json

# Step 1: Define a dummy function (in production, this would call a real API)
def get_current_weather(location, unit='fahrenheit'):
    """Get the current weather in a given location"""
    if 'tokyo' in location.lower():
        return json.dumps({'location': 'Tokyo', 'temperature': '10', 'unit': 'celsius'})
    elif 'san francisco' in location.lower():
        return json.dumps({'location': 'San Francisco', 'temperature': '72', 'unit': 'fahrenheit'})
    elif 'paris' in location.lower():
        return json.dumps({'location': 'Paris', 'temperature': '22', 'unit': 'celsius'})
    else:
        return json.dumps({'location': location, 'temperature': 'unknown'})

# Step 2: Define the function schema
functions = [{
    'name': 'get_current_weather',
    'description': 'Get the current weather in a given location',
    'parameters': {
        'type': 'object',
        'properties': {
            'location': {
                'type': 'string',
                'description': 'The city and state, e.g. San Francisco, CA',
            },
            'unit': {
                'type': 'string',
                'enum': ['celsius', 'fahrenheit']
            },
        },
        'required': ['location'],
    },
}]

print("✅ Function defined!")
print(f"Function: {functions[0]['name']}")
print(f"Description: {functions[0]['description']}")

In [ ]:
# Step 3: Create LLM client
llm = get_chat_model(llm_cfg)

# Step 4: Send user query with function definitions
messages = [{'role': 'user', 'content': "What's the weather like in San Francisco?"}]

print("User: What's the weather like in San Francisco?\n")
print("Calling LLM with function definitions...\n")

# Get LLM response
responses = []
for responses in llm.chat(
    messages=messages,
    functions=functions,
    stream=True
):
    pass  # Just get the final response

print("LLM Response:")
print(json.dumps(responses, indent=2, ensure_ascii=False))

### Understanding the Response

Notice the LLM response contains:
- `role`: 'assistant'
- `function_call`: A dict with `name` and `arguments`
- `arguments`: A JSON string (not a dict!)

Now let's execute the function and complete the interaction!

In [ ]:
# Step 5: Execute the function
messages.extend(responses)  # Add LLM's function call to history

last_response = messages[-1]
if last_response.get('function_call'):
    print("\n🔧 LLM requested function call!\n")
    
    # Parse the function call
    function_name = last_response['function_call']['name']
    function_args = json.loads(last_response['function_call']['arguments'])
    
    print(f"Function: {function_name}")
    print(f"Arguments: {function_args}\n")
    
    # Execute the function
    available_functions = {
        'get_current_weather': get_current_weather,
    }
    function_to_call = available_functions[function_name]
    function_response = function_to_call(
        location=function_args.get('location'),
        unit=function_args.get('unit', 'fahrenheit'),
    )
    
    print(f"Function Response: {function_response}\n")
    
    # Step 6: Add function result to messages
    messages.append({
        'role': 'function',
        'name': function_name,
        'content': function_response,
    })
    
    # Step 7: Call LLM again to get final answer
    print("Calling LLM again with function result...\n")
    final_responses = []
    for final_responses in llm.chat(
        messages=messages,
        functions=functions,
        stream=True
    ):
        pass
    
    print("Final Answer:")
    print(final_responses[-1].get('content', ''))

---
## Part 4: Function Call Prompt Types

### What is fncall_prompt_type?

Different models expect different function calling formats:

| Type | Description | When to Use |
|------|-------------|-------------|
| **'qwen'** | Qwen's native format | Qwen models via DashScope |
| **'nous'** | NousResearch format | Most OpenAI-compatible APIs |

**For Fireworks API:** We should use **'nous'** format!

Let's test both:

In [ ]:
# Test with 'nous' format (recommended for Fireworks)
llm_with_nous = get_chat_model({
    **llm_cfg,
    'generate_cfg': {
        **llm_cfg.get('generate_cfg', {}),
        'fncall_prompt_type': 'nous'
    }
})

messages = [{'role': 'user', 'content': "What's the weather in Paris?"}]

print("Testing with fncall_prompt_type='nous':\n")
for responses in llm_with_nous.chat(messages=messages, functions=functions, stream=True):
    pass

if responses and responses[-1].get('function_call'):
    print("✅ Function call detected!")
    print(f"   Function: {responses[-1]['function_call']['name']}")
    print(f"   Args: {responses[-1]['function_call']['arguments']}")
else:
    print("❌ No function call (might need different format)")

---
## Part 5: Controlling Function Calls with function_choice

### The function_choice Parameter

You can control when the LLM uses functions:

| Value | Behavior | Use Case |
|-------|----------|----------|
| **'auto'** (default) | LLM decides | Normal operation |
| **'none'** | Never call functions | Force direct answer |
| **function_name** | Force this function | Required tool use |

Let's see examples:

In [ ]:
# Example 1: Auto (default) - LLM decides
print("Example 1: function_choice='auto'\n")
messages = [{'role': 'user', 'content': "What's the weather in Tokyo?"}]

for responses in llm.chat(
    messages=messages,
    functions=functions,
    stream=True,
    extra_generate_cfg={'function_choice': 'auto'}
):
    pass

print(f"LLM decision: {'Call function' if responses[-1].get('function_call') else 'Direct answer'}")
print()

In [ ]:
# Example 2: Force function call
print("Example 2: Forcing function call\n")
messages = [{'role': 'user', 'content': "Tell me about Tokyo"}]

for responses in llm.chat(
    messages=messages,
    functions=functions,
    stream=True,
    extra_generate_cfg={'function_choice': 'get_current_weather'}
):
    pass

if responses[-1].get('function_call'):
    print("✅ Function was forced to be called!")
    print(f"   Arguments: {responses[-1]['function_call']['arguments']}")
print()

In [ ]:
# Example 3: Disable function calls
print("Example 3: function_choice='none' (disable functions)\n")
messages = [{'role': 'user', 'content': "What's the weather in Paris?"}]

for responses in llm.chat(
    messages=messages,
    functions=functions,
    stream=True,
    extra_generate_cfg={'function_choice': 'none'}
):
    pass

print(f"Has function_call: {responses[-1].get('function_call') is not None}")
print(f"Direct answer: {responses[-1].get('content', '')[:100]}...")

---
## Part 6: Parallel Function Calls

### Calling Multiple Functions at Once

When a user asks about multiple things, the LLM can call multiple functions in parallel!

Example: "What's the weather in San Francisco, Tokyo, and Paris?"

The LLM can generate 3 function calls in one response!

In [ ]:
# Enable parallel function calls
messages = [{
    'role': 'user',
    'content': "What's the weather like in San Francisco, Tokyo, and Paris?"
}]

print("User: What's the weather in San Francisco, Tokyo, and Paris?\n")
print("Calling LLM with parallel_function_calls=True...\n")

responses = []
for responses in llm.chat(
    messages=messages,
    functions=functions,
    stream=True,
    extra_generate_cfg={'parallel_function_calls': True}
):
    pass

# Check if we got multiple function calls
fncall_msgs = [rsp for rsp in responses if rsp.get('function_call')]
print(f"Number of function calls: {len(fncall_msgs)}\n")

for i, msg in enumerate(fncall_msgs, 1):
    print(f"Call {i}:")
    print(f"  Function: {msg['function_call']['name']}")
    print(f"  Arguments: {msg['function_call']['arguments']}")
    print()

### Executing Parallel Function Calls

When you get multiple function calls, execute them and add ALL results back:

In [ ]:
messages.extend(responses)  # Add all function calls to history

if fncall_msgs:
    print("Executing all function calls...\n")
    
    available_functions = {
        'get_current_weather': get_current_weather,
    }
    
    # Execute each function call
    for msg in fncall_msgs:
        function_name = msg['function_call']['name']
        function_args = json.loads(msg['function_call']['arguments'])
        
        function_to_call = available_functions[function_name]
        function_response = function_to_call(
            location=function_args.get('location'),
            unit=function_args.get('unit', 'fahrenheit'),
        )
        
        print(f"Result for {function_args['location']}: {function_response}")
        
        # Add function result to messages (in same order as calls!)
        messages.append({
            'role': 'function',
            'name': function_name,
            'content': function_response,
        })
    
    print("\nCalling LLM again with all results...\n")
    
    # Get final synthesized answer
    final_responses = []
    for final_responses in llm.chat(
        messages=messages,
        functions=functions,
        stream=True,
        extra_generate_cfg={'parallel_function_calls': True}
    ):
        pass
    
    print("Final Answer:")
    print(final_responses[-1].get('content', ''))

---
## Part 7: Error Handling

### Common Errors in Function Calling

1. **Malformed JSON** - LLM generates invalid JSON arguments
2. **Unknown function** - LLM tries to call a function you didn't define
3. **Missing required parameters** - LLM omits required arguments
4. **Type errors** - LLM provides wrong type (string instead of number)

Let's handle these gracefully:

In [ ]:
def safe_execute_function(function_call_msg, available_functions):
    """
    Safely execute a function call with error handling
    """
    try:
        function_name = function_call_msg['function_call']['name']
        
        # Check if function exists
        if function_name not in available_functions:
            return json.dumps({
                'error': f"Function '{function_name}' not found",
                'available_functions': list(available_functions.keys())
            })
        
        # Parse arguments
        try:
            function_args = json.loads(function_call_msg['function_call']['arguments'])
        except json.JSONDecodeError as e:
            return json.dumps({
                'error': f"Invalid JSON in arguments: {str(e)}",
                'arguments': function_call_msg['function_call']['arguments']
            })
        
        # Execute function
        function_to_call = available_functions[function_name]
        result = function_to_call(**function_args)
        return result
        
    except TypeError as e:
        return json.dumps({
            'error': f"TypeError: {str(e)}",
            'hint': 'Check if all required parameters are provided with correct types'
        })
    except Exception as e:
        return json.dumps({
            'error': f"Unexpected error: {str(e)}",
            'type': type(e).__name__
        })

# Test error handling
print("Testing error handling...\n")

# Simulate a malformed function call
test_msg = {
    'function_call': {
        'name': 'unknown_function',
        'arguments': '{"invalid": json}'
    }
}

result = safe_execute_function(test_msg, {'get_current_weather': get_current_weather})
print(f"Error handling result:\n{result}")

---
## Part 8: Multiple Functions Example

### Giving the LLM Multiple Tools

Let's add more functions for the LLM to choose from!

In [ ]:
# Define multiple functions
def get_current_time(timezone='UTC'):
    """Get the current time in a timezone"""
    from datetime import datetime
    return json.dumps({'timezone': timezone, 'time': datetime.now().isoformat()})

def calculate(expression):
    """Calculate a mathematical expression"""
    try:
        result = eval(expression)  # In production, use a safe math parser!
        return json.dumps({'expression': expression, 'result': result})
    except Exception as e:
        return json.dumps({'error': str(e)})

# Define function schemas
multi_functions = [
    {
        'name': 'get_current_weather',
        'description': 'Get the current weather in a given location',
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {'type': 'string', 'description': 'The city name'},
                'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}
            },
            'required': ['location']
        }
    },
    {
        'name': 'get_current_time',
        'description': 'Get the current time in a specific timezone',
        'parameters': {
            'type': 'object',
            'properties': {
                'timezone': {'type': 'string', 'description': 'Timezone like UTC, EST, PST'}
            },
            'required': []
        }
    },
    {
        'name': 'calculate',
        'description': 'Calculate a mathematical expression',
        'parameters': {
            'type': 'object',
            'properties': {
                'expression': {'type': 'string', 'description': 'Math expression like "2+2" or "sqrt(16)"'}
            },
            'required': ['expression']
        }
    }
]

available_functions = {
    'get_current_weather': get_current_weather,
    'get_current_time': get_current_time,
    'calculate': calculate
}

print(f"✅ Defined {len(multi_functions)} functions for the LLM!")

In [ ]:
# Test with a query that needs multiple functions
messages = [{
    'role': 'user',
    'content': 'What time is it and what is 15 * 23?'
}]

print("User: What time is it and what is 15 * 23?\n")

responses = []
for responses in llm.chat(
    messages=messages,
    functions=multi_functions,
    stream=True,
    extra_generate_cfg={'parallel_function_calls': True}
):
    pass

# Execute any function calls
fncall_msgs = [rsp for rsp in responses if rsp.get('function_call')]
if fncall_msgs:
    print(f"LLM requested {len(fncall_msgs)} function call(s):\n")
    
    messages.extend(responses)
    
    for msg in fncall_msgs:
        result = safe_execute_function(msg, available_functions)
        print(f"Function: {msg['function_call']['name']}")
        print(f"Result: {result}\n")
        
        messages.append({
            'role': 'function',
            'name': msg['function_call']['name'],
            'content': result
        })
    
    # Get final answer
    for final_responses in llm.chat(messages=messages, functions=multi_functions, stream=True):
        pass
    
    print("Final Answer:")
    print(final_responses[-1].get('content', ''))

---
## Part 9: Complete Function Calling Example

### Building a Complete Chat Loop

Let's build a complete function-calling chat loop that handles everything automatically!

In [ ]:
def function_calling_chat(user_query, functions, available_functions, max_turns=5):
    """
    Complete function calling loop
    """
    messages = [{'role': 'user', 'content': user_query}]
    llm = get_chat_model(llm_cfg)
    
    for turn in range(max_turns):
        print(f"\n--- Turn {turn + 1} ---")
        
        # Call LLM
        responses = []
        for responses in llm.chat(
            messages=messages,
            functions=functions,
            stream=True,
            extra_generate_cfg={'parallel_function_calls': True}
        ):
            pass
        
        messages.extend(responses)
        
        # Check for function calls
        fncall_msgs = [rsp for rsp in responses if rsp.get('function_call')]
        
        if not fncall_msgs:
            # No function calls - we have the final answer
            print("✅ Got final answer")
            return responses[-1].get('content', '')
        
        # Execute function calls
        print(f"Executing {len(fncall_msgs)} function(s)...")
        for msg in fncall_msgs:
            fn_name = msg['function_call']['name']
            print(f"  - {fn_name}")
            
            result = safe_execute_function(msg, available_functions)
            messages.append({
                'role': 'function',
                'name': fn_name,
                'content': result
            })
    
    return "Max turns reached"

# Test it!
answer = function_calling_chat(
    "What's the weather in Paris and what is 100 divided by 4?",
    multi_functions,
    available_functions
)

print("\n" + "="*50)
print("Final Answer:")
print(answer)

---
## Part 10: Practice Exercises

Now it's your turn!

### Exercise 1: Add a Translation Function

Create a `translate_text` function that:
- Takes `text` and `target_language` parameters
- Returns a dummy translation
- Define its schema
- Test it with the LLM

In [ ]:
# TODO: Implement translate_text function and schema
# Hint: Follow the pattern of get_current_weather

# Your code here:
# ...

### Exercise 2: Handle Parallel Weather Queries

Modify the weather function to handle 5+ cities in parallel.
Test with: "Compare weather in New York, London, Tokyo, Sydney, and Mumbai"

In [ ]:
# TODO: Test parallel weather queries
# Your code here:
# ...

### Exercise 3: Build a Unit Converter

Create a function calling system with:
- `convert_temperature(value, from_unit, to_unit)`
- `convert_length(value, from_unit, to_unit)`
- `convert_weight(value, from_unit, to_unit)`

Test with: "Convert 100 fahrenheit to celsius, 10 kilometers to miles, and 5 pounds to kilograms"

In [ ]:
# TODO: Implement unit converter system
# Your code here:
# ...

### Exercise 4: Error Recovery

Create a function that sometimes fails, and implement error recovery:
- If function returns error, add error to messages
- Ask LLM to try again with different parameters
- Limit retry attempts

In [ ]:
# TODO: Implement error recovery
# Your code here:
# ...

---
## Summary: What You Learned Today

### Core Concepts

✅ **Function calling** - LLMs generate structured function requests

✅ **Function schemas** - JSON Schema format for defining functions

✅ **Direct LLM calling** - Function calling without agents (using llm.chat())

✅ **fncall_prompt_type** - 'qwen' vs 'nous' formats

✅ **function_choice** - Control when functions are called ('auto', 'none', or function name)

✅ **Parallel function calls** - Multiple tools in one response

✅ **Error handling** - Gracefully handling malformed calls and execution errors

### Key Takeaways

1. **LLMs don't execute functions** - they generate requests for you to execute
2. **Function descriptions matter** - they teach the LLM when to use each tool
3. **Always validate** - Parse JSON carefully and handle errors
4. **Parallel calling is powerful** - Enable it for better UX
5. **The loop is important** - LLM → Function → LLM (repeat as needed)

### What's Next?

**Tomorrow (Day 7)**: We'll learn **Custom Tools**!

You'll learn:
- Using @register_tool decorator
- Parameter schema deep dive
- Building reusable tools
- Advanced tool patterns
- Tool testing

### Additional Resources

- 📖 Function Calling Example: `/examples/function_calling.py`
- 🔧 Parallel Calls Example: `/examples/function_calling_in_parallel.py`
- 💡 OpenAI Function Calling Guide: https://platform.openai.com/docs/guides/function-calling

---

**Congratulations! 🎉 You now understand how LLMs use tools through function calling!**